In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [37]:

driver = webdriver.Chrome()

# Navigate to the Aritzia Dresses page
driver.get("https://www.aritzia.com/en/clothing/jeans")

# Wait for the page to load (adjust time if needed)
wait = WebDriverWait(driver, 15)
# wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lazy")))

# Define the number of times to scroll
scroll_count = 30

# Simulate continuous scrolling using JavaScript
for _ in range(scroll_count):
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(2)  # Wait for new content to load




# Get the page source after scrolling
page_source = driver.page_source

# Close the WebDriver
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

# Find all product containers (modify if the structure changes)
product_containers = soup.find_all("li", class_="ar-product-grid__tile")



# List to store extracted product data
products = []


# Loop through each product container
for index, product in enumerate(product_containers):
    print(f"\nProcessing product {index + 1}...")  # Debugging info

    try:
        # **Extract Title** (From the <a> tag with the `title` attribute)
        title_tag = product.find("a", title=True)
        title = title_tag["title"].strip() if title_tag else "No Title"
        print(f"Title: {title}")  # Debugging info

        # **Extract Image URL** (Handle lazy loading with multiple attributes)
        image_tag = product.find("img")
        # image_url = (
        image_url = image_tag.get("data-mouseover-img")  #or  # Best for lazy-loaded images
            # image_tag.get("src") or  # Fallback if image is fully loaded
            # image_tag.get("data-mouseout-img") or  # Alternative for product images
            # (image_tag.get("data-srcset").split(",")[0].split(" ")[0] if image_tag and image_tag.get("data-srcset") else None)
            # if image_tag else "No Image")

        print(f"Image URL: {image_url}")  # Debugging info

        # **Extract Product Link**
        product_link = title_tag["href"] if title_tag and title_tag.has_attr("href") else "No Link"
        print(f"Product Link: {product_link}")  # Debugging info

        # **Extract Price** (Ensure correct class)
        price_span = product.find("span", class_="js-product__sales-price")
        price = price_span.get_text(strip=True) if price_span else "Price not found"
        print(f"Price: {price}")  # Debugging info

        # Append extracted data
        products.append({"Title": title, "Product Link": product_link, "Image URL": image_url, "Price": price})

    except Exception as e:
        print(f"Error extracting product {index + 1}: {e}")



Processing product 1...
Title: THE FARRAH HI-RISE WIDE JEAN
Image URL: https://assets.aritzia.com/image/upload/medium/s25_a06_109082_30426_on_b.jpg
Product Link: https://www.aritzia.com/en/product/the-farrah-hi-rise-wide-jean/109082.html?dwvar_109082_color=30426
Price: $128

Processing product 2...
Title: ANNINA JEAN
Image URL: https://assets.aritzia.com/image/upload/medium/s25_a06_113877_31768_on_b.jpg
Product Link: https://www.aritzia.com/en/product/annina-jean/113877.html?dwvar_113877_color=31768
Price: $338-$368

Processing product 3...
Title: THE '90S IGGY LO-RISE BAGGY JEAN
Image URL: https://assets.aritzia.com/image/upload/medium/s25_a06_107109_30426_on_b.jpg
Product Link: https://www.aritzia.com/en/product/the-%2790s-iggy-lo-rise-baggy-jean/107109.html?dwvar_107109_color=30426
Price: $128

Processing product 4...
Title: AYLA BAGGY CUFFED JEAN
Image URL: https://assets.aritzia.com/image/upload/medium/s25_a06_110210_30959_on_b.jpg
Product Link: https://www.aritzia.com/en/product

In [38]:
# Convert extracted data into a Pandas DataFrame
jeans_df = pd.DataFrame(products)

# Save data to CSV 
jeans_df.to_csv("aritzia_jeans_v2.csv", index=False)
print("Data saved to aritzia_jeans_v2.csv")


Data saved to aritzia_jeans_v2.csv
